#**Install Pinecone**

In [ ]:
%pip install -qU langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.7 MB/s eta 0:00:00


#**Pinecone API Key**

In [ ]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key =userdata.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

# **Create Index**

In [ ]:
import time

index_name = "langchain-rag-project"  # change if desired

pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)
index = pc.Index(index_name)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '6aecc25047cb4bef897cfffcf9948738', 'Date': 'Mon, 06 Jan 2025 15:13:25 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


# **Install google-genai**

In [ ]:
%pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00


# **Set Up Embedding Model**

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

import os
os.environ['GOOGLE_API_KEY'] =userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vector = embeddings.embed_query("what is Python?")

vector[:5]

[0.02367168292403221,
 -0.05139750987291336,
 -0.007409077603369951,
 -0.050910450518131256,
 0.03173977509140968]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index,embedding=embeddings)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [ ]:
len(documents)

10

In [ ]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['4d1f84ff-a97e-4443-b1f0-35fc6a3fca00',
 'b1edcca7-12e2-45d3-8fa4-5a7422e95524',
 '4fbb75ac-a2ce-437e-a8fb-1ca14a0dc2b2',
 '7bb7687e-d22e-4afd-bf97-dbe7e3f49580',
 '937a4ae8-302b-4444-9933-b20c6692bece',
 'd1431632-e8cb-494b-8d7c-b7aebce33da9',
 '62c054cd-9761-41b9-b5e7-e335aa7fd757',
 '315afcd2-0587-4cfe-ba9a-2cb772e9a4ce',
 '591320b0-8123-4e80-9b36-8e771fb07ec3',
 '77656eef-a05d-4f73-aafa-24f794501e80']

In [ ]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:

    print(f"* {res.page_content} [{res.metadata}]")

In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

In [ ]:
def answer_to_user(query:str):
  #vector search
  result = vector_store.similarity_search(query,k=2)

  #pass to  model search
  final_answer=model(results,query)
  return final_answer